# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle

# TODO: Fill this in based on where you saved the training and testing data

dir_name = "traffic-signs-data/"
training_file   = dir_name + "train.p"
validation_file = dir_name + "valid.p"
testing_file    = dir_name + "test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

assert(len(X_train) == len(y_train))
assert(len(X_valid) == len(y_valid))
assert(len(X_test) == len(y_test))

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

# TODO: Number of training examples
n_train = len(X_train)

# TODO: Number of validation examples
n_validation = len(X_valid)

# TODO: Number of testing examples.
n_test = len(X_test)

# TODO: What's the shape of an traffic sign image?
image_shape = X_train[0].shape

# TODO: How many unique classes/labels there are in the dataset.
labels = []
for label in y_train:
    if label not in labels:
        labels.append(label)
n_classes = len(labels)

print("Number of training examples =", n_train)
print("Number of validation examples =", n_validation)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)
print("Train dataset shape =", X_train.shape)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline

# show 5 ramdom inages
import random
import pandas as pd

# Utility to plot random images
def plot_images(images, lables, n = 5, cmap=None, idx_random = True):
    signnames = pd.read_csv('./signnames.csv').values
    figure, axarr = plt.subplots(1, n, figsize=(17,3),sharex=False, sharey=False)
    for subplot in range(n):
        if idx_random:
            index = random.randint(0, len(images)-1)
        else:
            index = subplot
        image = images[index].squeeze()
        axarr[subplot].imshow(image, cmap=cmap)
        lbl = lables[index]
        axarr[subplot].set_title(signnames[lbl][1], {'fontsize': 8})
    plt.show()
    return figure

print()
print("Examples of the Traffic Sing training images:")
figure = plot_images(X_train, y_train)

# Save the samples (for report)
figure.savefig('project_data/color_signs.png')   # save the figure to file
plt.close(figure)    # close the figure

# Histogram of number of images of each sign
hist = plt.figure()
plt.hist(y_train, bins='auto')
plt.title("Destribution of each sign in the training data set")
plt.show()

# Save histogram picture (for report)
hist.savefig('project_data/hist.png')   # save the histogram to file
plt.close(hist)    # close the figure


----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed.

## Utility functions

# Returns all indices in train dataset for a specific class 
def idx_in_class(s_class):
    signs = [] 
    for i in range(n_train):
        if s_class == y_train[i]:
            signs.append(i)
    return signs


Grayscale by simply averaging the color pixels: (R + G + B) / 3 ([source](https://www.johndcook.com/blog/2009/08/24/algorithms-convert-color-grayscale/))

In [ ]:
import numpy as np

X_train = np.sum(X_train, axis=3, keepdims=True)/3
X_test = np.sum(X_test, axis=3, keepdims=True)/3
X_valid = np.sum(X_valid, axis=3, keepdims=True)/3


In [ ]:
print()
print("Examples of the preprocessed images:")
figure = plot_images(X_train, y_train, cmap='gray')

# Save the grayscale samples (for report)
figure.savefig('project_data/gray_signs.png')   # save the figure to file
plt.close(figure)    # close the figure


Dataset Augmentation - adding more data to the training dataset by flipping and rotating the sign images 

In [ ]:
# flip
def flip_signs(x, y, signs, num=10):
    for i in range(num):
        idx = np.random.choice(signs)
        image = np.flipud(X_train[idx])
        x.append(image)
        y.append(y_train[idx])
    return x, y

# rotation
import scipy.ndimage as ndimage

def rotate_signs(x, y, signs, angle=15.0, num=10):
    for i in range(num):
        idx = np.random.choice(signs)
        image = ndimage.rotate(X_train[idx], random.random()*angle, reshape=False)
        x.append(image)
        y.append(y_train[idx])
    return x, y

# shift (affine warp)
from skimage.transform import warp, AffineTransform

def shift_signs(x, y, signs, num):
    for i in range(num):
        idx = np.random.choice(signs)
        dx, dy, ang, shr = (np.random.random(4) - 0.5)/10.
        #tform = AffineTransform(scale=(dx+1, dy+1), shear=None, rotation=None)
        tform = AffineTransform(scale=(dx+1, dy+1), shear=shr*2.5, rotation=ang*2.5)
        image = warp(X_train[idx], tform.inverse)
        x.append(image)
        y.append(y_train[idx])
    return x, y

# tilt (projective warp)
from skimage.transform import ProjectiveTransform

def tilt_signs(x, y, signs, num):
    for i in range(num):
        idx = np.random.choice(signs)
        high = random.randint(27, 32)
        low = random.randint(0, 5)
        p1 = np.float32([[0,0],[32,0],[0,32],[32,32]])
        p2 = np.float32([[0, 0], [high, low], [low, 32],[32, high]])
        tform = ProjectiveTransform()
        tform.estimate(p1, p2)  
        image = warp(X_train[idx], tform.inverse)
        x.append(image)
        y.append(y_train[idx])
    return x, y    

In [ ]:
# Build distorted signes for all classes, on 20% per of signes in class
X_flip = []
y_flip = []
X_left = []
y_left = []
X_right = []
y_right = []
X_shift = []
y_shift = []
X_tilt = []
y_tilt = []


for i in range(n_classes):
    signs = idx_in_class(i)
    num = len(signs)//5
    
    #flip
    flip_signs( X_flip, y_flip, signs, num)

    #rotate left
    rotate_signs( X_left, y_left, signs, 15.0, num)

    #rotate right
    rotate_signs( X_right, y_right, signs, -15.0, num)

    #shift
    shift_signs( X_shift, y_shift, signs, num)
    
    #tilt
    tilt_signs( X_tilt, y_tilt, signs, num)

# convert from list
X_flip = np.array(X_flip)
y_flip = np.array(y_flip)
X_left = np.array(X_left)
y_left = np.array(y_left)
X_right = np.array(X_right)
y_right = np.array(y_right)
X_shift = np.array(X_shift)
y_shift = np.array(y_shift)
X_tilt = np.array(X_tilt)
y_tilt = np.array(y_tilt)



In [ ]:
print()
print("Examples of the flipped images:")
figure = plot_images(X_flip, y_flip, cmap='gray')
# Save
figure.savefig('project_data/flip_signs.png')   # save the figure to file
plt.close(figure)    # close the figure

In [ ]:
print()
print("Examples of the left rotated images:")
figure = plot_images(X_left, y_left, cmap='gray')
# Save
figure.savefig('project_data/left_signs.png')   # save the figure to file
plt.close(figure)    # close the figure

In [ ]:
print()
print("Examples of the right rotated images:")
figure = plot_images(X_right, y_right, cmap='gray')
# Save
figure.savefig('project_data/right_signs.png')   # save the figure to file
plt.close(figure)    # close the figure

In [ ]:
print()
print("Examples of the shifted images:")
figure = plot_images(X_shift, y_shift, cmap='gray')
# Save
figure.savefig('project_data/shift_signs.png')   # save the figure to file
plt.close(figure)    # close the figure

In [ ]:
print()
print("Examples of the projective warp images:")
figure = plot_images(X_tilt, y_tilt, cmap='gray')
# Save
figure.savefig('project_data/proj_signs.png')   # save the figure to file
plt.close(figure)    # close the figure

In [ ]:
# Add new images to the training dataset
X_train = np.concatenate((X_train, X_left, X_right, X_shift, X_tilt), axis=0)
y_train = np.concatenate((y_train, y_left, y_right, y_shift, y_tilt), axis=0)

print(X_train.shape)
n_train = len(X_train)

Normalization

In [ ]:
#X_train = (X_train - 128)/128 
#X_test = (X_test - 128)/128
#X_valid = (X_valid - 128)/128

def normalize_grayscale(image_data):
    """
    Normalize the image data with Min-Max scaling to a range of [0.1, 0.9]
    :param image_data: The image data to be normalized
    :return: Normalized image data
    """
    # TODO: Implement Min-Max scaling for grayscale image data
    # scaled image X′ = a + ((X − Xmin)(b − a))/(Xmax − Xmin)
    # a = 0.1, b = 0.9, Xmin = 0, Xmax = 255
    # Calculate scale
    Xmin = 0
    Xmax = 255
    a = 0.1
    b = 0.9
    image_scaled = a + ((image_data - Xmin)* (b - a))/(Xmax - Xmin)
    return image_scaled

X_train = normalize_grayscale(X_train)
X_test = normalize_grayscale(X_test)
X_valid = normalize_grayscale(X_valid)


Shuffle the training data

In [ ]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

### Model Architecture

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.
from tensorflow.contrib.layers import flatten

def LeNet(x):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    # TODO: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x64.
    #new_height = (input_height - filter_height + 2 * P)/S + 1 = (32-5+2*0)/1+1=28
    #new_width = (input_width - filter_width + 2 * P)/S + 1 = (32-5+2*0)/1+1=28
    W1 = tf.Variable(tf.truncated_normal([5, 5, 1, 64], mean=mu, stddev=sigma)) 
    b1 = tf.Variable(tf.zeros(64))
    conv1 = tf.nn.conv2d(x, W1, strides=[1, 1, 1, 1], padding='VALID')
    conv1 = tf.nn.bias_add(conv1, b1)
    print("conv1 shape: {}".format(conv1))

    # TODO: Activation 1.
    act1 = tf.nn.relu(conv1)

    #act1 = tf.nn.dropout(act1, keep_prob)    

    # TODO: Pooling 1. Input = 28x28x64. Output = 14x14x64.
    # new_height = (input_height - filter_height)/S + 1 = (28 - 2)/2 +1 = 13+1 = 14
    # new_width = (input_width - filter_width)/S + 1 = (28 - 2)/2 +1 = 13+1 = 14
    pool1 = tf.nn.max_pool(act1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    print("pool1 Shape: {}".format(pool1))

    # TODO: Layer 2: Convolutional. Input = 14x14x64. Output = 12x12x96.
    #new_height = (input_height - filter_height + 2 * P)/S + 1 = (14-3+2*0)/1+1=12
    #new_width = (input_width - filter_width + 2 * P)/S + 1 = (14-3+2*0)/1+1=12
    W2 = tf.Variable(tf.truncated_normal([3, 3, 64, 96], mean=mu, stddev=sigma)) 
    b2 = tf.Variable(tf.zeros(96))
    conv2 = tf.nn.conv2d(pool1, W2, strides=[1, 1, 1, 1], padding='VALID')
    conv2 = tf.nn.bias_add(conv2, b2)
    print("conv2 Shape: {}".format(conv2))
    
    # TODO: Activation 2.
    act2 = tf.nn.relu(conv2)
    
    # TODO: Pooling 2. Input = 12x12x96. Output = 6x6x96.
    # new_height = (input_height - filter_height)/S + 1 = (12 -2)/2 +1 = 5+1 = 6
    # new_width = (input_width - filter_width)/S + 1 = (12 -2)/2 +1 = 5+1 = 6
    pool2 = tf.nn.max_pool(act2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    print("pool2 shape: {}".format(pool2))

    # TODO: Layer 2b: Convolutional. Input = 6x6x96. Output = 5x5x128.
    #new_height = (input_height - filter_height + 2 * P)/S + 1 = (6-2+2*0)/1+1=5
    #new_width = (input_width - filter_width + 2 * P)/S + 1 = (6-2+2*0)/1+1=5
    W2b = tf.Variable(tf.truncated_normal([2, 2, 96, 128], mean=mu, stddev=sigma)) 
    b2b = tf.Variable(tf.zeros(128))
    conv2b = tf.nn.conv2d(pool2, W2b, strides=[1, 1, 1, 1], padding='VALID')
    conv2b = tf.nn.bias_add(conv2b, b2b)
    print("conv2b Shape: {}".format(conv2b))
    
    # TODO: Activation.
    act2b = tf.nn.relu(conv2b)
    
    # TODO: Pooling. Input = 5x5x128. Output = 4x4x128.
    # new_height = (input_height - filter_height)/S + 1 = (5 -2)/1 +1 = 3+1 = 4
    # new_width = (input_width - filter_width)/S + 1 = (5 -2)/1 +1 = 3+1 = 4
    pool2b = tf.nn.max_pool(act2b, ksize=[1, 2, 2, 1], strides=[1, 1, 1, 1], padding='VALID')
    print("pool2b shape: {}".format(pool2b))
    
    # TODO: Flatten. Input = 4x4x128. Output = 2048.
    flat = flatten(pool2b)
    print("flat shape: {}".format(flat))
    
    # Dropout
    flat = tf.nn.dropout(flat, keep_prob)    

    # TODO: Layer 3: Fully Connected. Input = 2048. Output = 512.
    W3 = tf.Variable(tf.truncated_normal([2048, 512], mean=mu, stddev=sigma)) 
    b3 = tf.Variable(tf.zeros(512))
    fc3 = tf.add(tf.matmul(flat, W3), b3)
    print("fc3 shape: {}".format(fc3))
    
    # TODO: Activation.
    act3 = tf.nn.relu(fc3)

    # Dropout
    act3 = tf.nn.dropout(act3, keep_prob)    

    # TODO: Layer 4: Fully Connected. Input = 512. Output = 256.
    W4 = tf.Variable(tf.truncated_normal([512, 256], mean=mu, stddev=sigma)) 
    b4 = tf.Variable(tf.zeros(256))
    fc4 = tf.add(tf.matmul(act3, W4), b4)
    print("fc4 shape: {}".format(fc4))
    
    # TODO: Activation.
    act4 = tf.nn.relu(fc4)

    # Dropout
    act4 = tf.nn.dropout(act4, keep_prob)    

    # TODO: Layer 5: Fully Connected. Input = 256. Output = 43.
    W5 = tf.Variable(tf.truncated_normal([256, 43], mean=mu, stddev=sigma)) 
    b5 = tf.Variable(tf.zeros(43))
    logits = tf.add(tf.matmul(act4, W5), b5)
    print("fc4 shape: {}".format(logits))
    
    return logits

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

# Setup TensorFlow
import tensorflow as tf

EPOCHS = 0#50
BATCH_SIZE = 128

# x is a placeholder for a batch of input images. y is a placeholder for a batch of output labels.
x = tf.placeholder(tf.float32, (None, 32, 32, 1))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, 43)

# Create a training pipeline
rate = 0.001

# Probability to keep units
keep_prob = tf.placeholder(tf.float32)
dropout = 0.7

logits = LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

# Evaluate how well the loss and accuracy of the model for a given dataset
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.0})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

# Run the training data through the training pipeline to train the model
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
            
        validation_accuracy = evaluate(X_valid, y_valid)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './lenet')
    is_model_saved = True
    print("Model saved")

In [ ]:

with tf.Session() as sess:
    #saver.restore(sess, tf.train.latest_checkpoint('.'))
    saver.restore(sess, "./lenet")

    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))


---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.
import glob
import matplotlib.image as mpimg
import cv2
import tensorflow as tf
import numpy as np
import pandas as pd

new_files = glob.glob('images/*.jpg')
new_files = sorted(new_files)
new_labels = np.array([17, 25, 34, 3, 14])

new_images_origin = []
new_images = []

for file in new_files:
    # read from files
    image_origin = cv2.cvtColor(cv2.imread(file), cv2.COLOR_BGR2RGB)
    new_images_origin.append(image_origin)
    # resize
    image = cv2.resize(image_origin, (32, 32), interpolation = cv2.INTER_AREA)
    new_images.append(image)

new_images_origin = np.array(new_images_origin)
new_images = np.array(new_images)
print(new_images.shape)

# Utility to plot random images
def plot_images(images, lables, n = 5, cmap=None, idx_random = True):
    signnames = pd.read_csv('./signnames.csv').values
    #figure, axarr = plt.subplots(1, n, figsize=(17,3), sharex=True, sharey=True)
    figure, axarr = plt.subplots(1, n, figsize=(17,3), sharex=False, sharey=False)
    for subplot in range(n):
        if idx_random:
            index = random.randint(0, len(images)-1)
        else:
            index = subplot
        image = images[index].squeeze()
        axarr[subplot].imshow(image, cmap=cmap)
        lbl = lables[index]
        axarr[subplot].set_title(signnames[lbl][1], {'fontsize': 8})
    plt.show()
    return figure

print()
print("New Traffic Sign images:")
figure = plot_images(new_images_origin, new_labels, idx_random = False)


### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

In [ ]:
# Grayscale and normalize
new_images = np.sum(new_images, axis=3, keepdims=True)/3
#new_images = (new_images - 128)/128
new_images = normalize_grayscale(new_images)

print(new_images.shape)
print()
print("Pre-processed new images:")
figure = plot_images(new_images, new_labels, cmap='gray', idx_random = False)

In [ ]:
predict_op = tf.argmax(logits, 1)
#softmax = tf.nn.softmax(logits)

signnames = pd.read_csv('./signnames.csv').values
predictions = []

with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    saver.restore(sess, "./lenet")
    predictions = sess.run(predict_op, feed_dict={x: new_images, keep_prob: 1.0})
    print(predictions)
    print()
    print("Ptredicted images:\n")
    figure = plot_images(new_images_origin, predictions, idx_random = False)  


### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

In [ ]:
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    saver.restore(sess, "./lenet")
    #saver.restore(sess, tf.train.latest_checkpoint('.'))
    accuracy = evaluate(new_images, new_lables)
    print("New Data Set Accuracy = {:.3f}".format(accuracy))

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")